### Prediction vs Control
| Aspect      | Prediction                                      | Control                                 |
|-------------|-------------------------------------------------|-----------------------------------------|
| Goal        | Evaluate how good the current policy is         | Find the optimal policy                 |
| Question    | Given a policy π, what is the value of state s? | What policy yields the maximum return?  |
| Output      | Value function $V^\pi(s)$ or $Q^\pi(s, a)$ | Optimal policy $\pi^*$              |

### Model-Free vs Model-Based
Model-based: I know the rules of the game. When I take an action, I know the outcome.
Model-free: I don't know the rules of the game. I start from scratch and to directly to learn the behavior of the game.

### Monte Carlo Estimation of Returns
Monte Carlo: To know the area of an irregular shape, we can shy a lot of beans on it. The smaller the bean and the more beans we use, the more accurate the estimate will be.        
First-Visit MC: Only the first time a state is iterated is counted.     
   
`For` each episode:<br>
　　　`If` state $s$ is visited for the first time:<br>
　　　　　　`Append` $G$ to Returns$(s)$<br>
　　　　　　`Update`: $V(s) ← V(s) + α[G - V(s)]$<br>        

Every-Visit MC: Every time a state is iterated is counted.

`For` each episode:<br>
　　　`For` each state $s$:<br>
　　　　　　`Append` $G$ to Returns$(s)$<br>
　　　　　　`Update`: $V(s) ← V(s) + α[G - V(s)]$<br>

FVMC vs EVMC is another typical example of the efficiency-vs-accuracy trade-off in machine learning.

### Temporal-Difference vs Monte Carlo
| Feature                        | Temporal Difference (TD)                | Monte Carlo (MC)                      |
|---------------------------------|-----------------------------------------|---------------------------------------|
| Update Timing                  | Updates after every step (online)       | Updates after each episode (offline)  |
| Bootstrapping                  | Yes (uses estimated value of next state)| No (uses actual returns only)         |
| Sample Efficiency              | More sample efficient                   | Less sample efficient                 |
| Variance                       | Lower variance                          | Higher variance                       |
| Bias                           | May have bias due to bootstrapping      | Unbiased (if enough episodes)         |
| Need for Episode End           | No (can learn from incomplete episodes) | Yes (needs complete episodes)         |
| Example Algorithms             | SARSA, Q-learning, TD(0)                | First-Visit MC, Every-Visit MC        |
| Convergence Speed              | Usually faster                          | Usually slower                        |
| Applicability to Continuing Tasks | Yes                                  | No (typically for episodic tasks)     |

### Q-Learning
$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_t + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t) \right]
$$
Q-learning uses the maximum Q-value of the next state to update current action values, rather than the expected value under the actual policy. This optimistic assumption leads to overestimation bias.

##### Exploitation vs Exploration and $\epsilon$-greedy Policy
Exploration: Each episode updates Q-values for all actions. It reflects a combination of TD and MC philosophies.
Bootstrapping relies on the previous estimate of the Q-value. It's like people always sticks to the same path; the experience built up in the past can facilitate efficiency, but there will come up new problems that can't be solved with the old methods.      
$\epsilon$ can be regarded as how curious the agent is about the new path. If it's too high, the agent may forego the old path but it's not guaranteed to find another optimal path. So, $\epsilon$ is usually set to $0.1$ or even lower.     
Moreover, while the knowledge system becomes relatively complete, the probability of discovering something new gets lower, so $\epsilon$ reduces over steps, e.g. from $0.1$ to $0.01$ after 200 steps.
Exploration-Exploitation dilemma is very much like bias-variance trade-off in Deep Learning.



### Sarsa
$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_t + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t) \right]
$$
Sarsa usese the direct value of the next state to update the current action value.

##### On-Policy vs Off-Policy
Notably, the Sarsa algorithm generates data samples using the current policy during training and updates based on them. In other words, the policy evaluation and policy improvement processes are performed using the same policy, which is referred to as an on-policy algorithm. Correspondingly, algorithms such as Q-learning that obtain samples from other policies and use them to update the target policy are referred to as off-policy algorithms. In other words, off-policy algorithms essentially learn from experience pools or historical data.

In [ ]:
# 1. Define the training process

for i_ep in range(train_eps): # iterate over episodes
    # reset environment, get initial state
    state = env.reset()  # reset environment, i.e. start a new episode
    while True: # for complex games, we can set the maximum number of steps per episode,
                # e.g. while ep_step<100, i.e. the maximum number of steps is 100.
        # the agent samples an action according to the policy
        action = agent.sample_action(state)  # sample an action
        # interact with the environment, get the next state and reward
        next_state, reward, terminated, _ = env.step(action)  # agent records the sample to the experience pool
        agent.memory.push(state, action, reward, next_state, terminated) 
        # agent updates the policy
        agent.update(state, action, reward, next_state, terminated)  
        # update the state
        state = next_state  
        # if the episode is terminated, the current episode ends
        if terminated:
            break
        
        
# 2. Define the algorithm
# 2.1 Sample an action
class Agent:
    def __init__(self, n_actions):
        self.Q_table = defaultdict(lambda: np.zeros(n_actions))
        pass
    
    def sample_action(self, state):
        self.sample_count += 1
    
        # epsilon decreases exponentially
        self.epsilon = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * math.exp(- self.sample_count / self.epsilon_decay) 
        
        # e-greedy strategy
        if np.random.uniform(0, 1) > self.epsilon:
            action = np.argmax(self.Q_table[str(state)]) # select the action with the highest Q-value
        else:
            action = np.random.choice(self.n_actions) # randomly select an action
        return action
    
# 2.2 Predict an action
    def predict_action(self,state):
        action = np.argmax(self.Q_table[str(state)])
        return action
    
# 2.3 Update the Q-table
    def update(self, state, action, reward, next_state, terminated):
        Q_predict = self.Q_table[str(state)][action] 
        if terminated: # for a terminated state
            Q_target = reward  
        else:
            Q_target = reward + self.gamma * np.max(self.Q_table[str(next_state)]) 
        self.Q_table[str(state)][action] += self.lr * (Q_target - Q_predict)

In [ ]:
# 3. Prepare the environment
env = gym.make('CliffWalking-v0') # by OpenAI Gym
n_states = env.observation_space.n # number of states
n_actions = env.action_space.n # number of actions
print(f"Number of states: {n_states}, number of actions: {n_actions}")

In [ ]:
# 4. Set parameters
self.epsilon_start = 0.95 #  e-greedy策略中epsilon的初始值
self.epsilon_end = 0.01 #  e-greedy策略中epsilon的最终值
self.epsilon_decay = 200 #  e-greedy策略中epsilon的衰减率

In [ ]:
# 5. Ablation study
# set the initial and final values to be the same, so that epsilon won't decay
self.epsilon_start = 0.1
self.epsilon_end = 0.1
self.epsilon_decay = 200